# **Análisis de los residuos para cada volatilidad**

## **Librerías**

In [1]:
import pandas as pd
import joblib
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# Carpeta de modelos relativa al notebook
model_dir = os.path.join(os.getcwd(), '..', 'app', 'modelos_finales')

if not os.path.exists(model_dir):
    raise FileNotFoundError(f"No se encontró el directorio {model_dir}")

# Listar archivos .joblib
model_files = [f for f in os.listdir(model_dir) if f.endswith('.joblib')]
print("Archivos encontrados:", model_files)

# Cargar los modelos
modelos_cargados = {f: joblib.load(os.path.join(model_dir, f)) for f in model_files}
print(f"Se cargaron {len(modelos_cargados)} modelos.")


Archivos encontrados: ['mejor_modelo_vol14d.joblib', 'mejor_modelo_vol21d.joblib', 'mejor_modelo_vol28d.joblib', 'mejor_modelo_vol7d.joblib']
Se cargaron 4 modelos.


In [3]:
# Diccionario para guardar residuos de todos los modelos
residuos_modelos = {}

for nombre, modelo_data in modelos_cargados.items():
    residuos_modelos[nombre] = modelo_data['residuos_mejor_fold']


## **Test BDS: Primer horizonte**

* **Hipótesis Nula ($H_0$)**: Los datos son **independientes e idénticamente distribuidos (i.i.d.)**.

    * El modelo ha hecho un buen trabajo.

* **Hipótesis Alternativa ($H_1$)**: Los datos **no son i.i.d.**

    * Esto sugiere que los residuos todavía contienen una estructura o patrón predecible (dependencia no lineal) que el modelo no logró capturar.

##### **Interpretación del p-valor:**
* Si el **p-valor ≥ 0.05**, no se rechaza la hipótesis nula ($H_0$). Se concluye que los residuos son **i.i.d.** (resultado deseado).
* Si el **p-valor < 0.05**, se rechaza la hipótesis nula ($H_0$). Se concluye que existe **dependencia** en los residuos.

In [7]:
BASE_DIR = os.getcwd()
results_path = os.path.join(BASE_DIR, 'results')
os.makedirs(results_path, exist_ok=True)

# Crear lista para almacenar la información de cada modelo
resumen_bds = []

for nombre, modelo_data in modelos_cargados.items():
    volatilidad = modelo_data['parametros']['volatilidad']
    lag = modelo_data['parametros']['lag']
    fold = modelo_data['parametros']['fold']
    bds = modelo_data['metricas']['BDS_pvalue']
    
    resumen_bds.append({
        'Modelo': f'Volatilidad {volatilidad} días',
        'Lag': lag,
        'Fold': fold,
        'BDS p-value H1': bds
    })

# Convertir a DataFrame
df_bds = pd.DataFrame(resumen_bds)

# Ordenar por lag y fold
df_bds.to_csv(os.path.join(results_path, 'resumen_bds.csv'), index=False)

# Mostrar tabla
df_bds


,Modelo,Lag,Fold,BDS p-value H1
0,Volatilidad 14 días,14,4,0.15807113740669199
1,Volatilidad 21 días,21,3,0.6527340508319626
2,Volatilidad 28 días,14,4,0.9586437812573544
3,Volatilidad 7 días,7,1,0.3573422471776194


Los 4 modelos presentan **p-valores > 0.05** en el test BDS para el primer horizonte (H1), lo que indica que **no se rechaza la hipótesis de independencia de los residuos**. Esto sugiere que los residuos se comportan como un proceso **iid**, lo que es consistente con un buen ajuste del modelo para el primer paso de predicción.


## **Medidas de tendencia**

In [5]:
BASE_DIR = os.getcwd()  # Esto toma la carpeta actual de ejecución

results_path = os.path.join(BASE_DIR, 'results')
os.makedirs(results_path, exist_ok=True)

resumen_residuos = []

for nombre, residuos in residuos_modelos.items():
    # Calcular media y desviación estándar de todos los residuos del fold
    media = np.mean(residuos)
    std = np.std(residuos)
    
    # Extraer volatilidad del nombre del modelo
    vol = int(nombre.split('_vol')[1].split('d')[0])
    
    resumen_residuos.append({
        'Modelo': f'Volatilidad {vol} días',
        'Media_Residuos': media,
        'Std_Residuos': std
    })

# Crear DataFrame final
df_resumen_residuos = pd.DataFrame(resumen_residuos)
df_resumen_residuos.set_index('Modelo', inplace=True)

# Guardar en CSV
df_resumen_residuos.to_csv(os.path.join(results_path, 'resumen_residuos.csv'))

# Mostrar tabla
df_resumen_residuos

,Media_Residuos,Std_Residuos
Modelo,,
Volatilidad 14 días,-0.011939,0.146753
Volatilidad 21 días,0.018149,0.108492
Volatilidad 28 días,-0.004625,0.076749
Volatilidad 7 días,0.013726,0.215244


Al observar los valores de la media y la desviación estándar de los residuos por modelo, se puede notar que todas las medias están muy cerca de cero, lo que indica que las predicciones, en promedio, no tienden a sobreestimar ni subestimar los valores reales de volatilidad. Por otro lado, la desviación estándar muestra que los modelos con ventanas más cortas (como 7 días) presentan mayor dispersión de residuos comparado con los modelos con ventanas más largas (como 28 días). 